# LLM-Based Eng-to-Prsn Translation System Using In-Context Learning

1. Model Choice (Light LLaMA-Based)

Recommended lightweight LLaMA-family models on Hugging Face:

✅ TinyLLaMA/TinyLLaMA-1.1B-Chat-v1.0
✅ NousResearch/Llama-2-7b-chat-hf (if you have GPU ≥16GB)

For a light notebook, we’ll use:

## Install dependencies

In [1]:
!pip install transformers accelerate sentencepiece sacrebleu datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.8 MB/s eta 0:00:00


In [3]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `LLMCourse` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticat

## Import libraries

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import sacrebleu

## Load the Pretrained LLaMA Model

In [4]:
model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=False
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

model.eval()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
   

## Dataset (English → Persian)


*   opus100 (en–fa)

*   Tatoeba

*   Custom CSV

In [5]:
dataset = load_dataset("opus100", "en-fa", split="test[:100]")

README.md: 0.00B [00:00, ?B/s]

en-fa/test-00000-of-00001.parquet:   0%|          | 0.00/193k [00:00<?, ?B/s]

en-fa/train-00000-of-00001.parquet:   0%|          | 0.00/82.4M [00:00<?, ?B/s]

en-fa/validation-00000-of-00001.parquet:   0%|          | 0.00/190k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [6]:
len(dataset)

100

In [7]:
dataset[0]["translation"]


{'en': "I'm obsolete!", 'fa': 'من دیگه به دردی نمیخورم'}

## In-Context Learning Prompt Design

Prompt Template

few-shot examples are provided inside the prompt.

In [8]:
def build_prompt(examples, source_sentence):
    prompt = "Translate the following sentences from English to Persian.\n\n"

    for ex in examples:
        prompt += f"English: {ex['en']}\n"
        prompt += f"Persian: {ex['fa']}\n\n"

    prompt += f"English: {source_sentence}\nPersian:"
    return prompt

In [9]:
n = 5
icl_examples = dataset[0:n]["translation"]

In [11]:
icl_examples

[{'en': "I'm obsolete!", 'fa': 'من دیگه به دردی نمیخورم'},
 {'en': "You've really gotta help me now.",
  'fa': 'تو واقعا ديگه بايد کمکم کني.'},
 {'en': 'The entire time you were in rehab,',
  'fa': '\u200fکل مدت\u200cزمانی که توی بازپروری بودی'},
 {'en': 'How well do you know this Dante ?',
  'fa': 'چقدر اين دانته رو ميشناسي؟'},
 {'en': 'I left my other clothes inside.', 'fa': 'لباس قبلي هامو جا گذاشتم'}]

## Translation Function

In [13]:
def translate(sentence, examples):
    prompt = build_prompt(examples, sentence)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=False
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded.split("Persian:")[-1].strip()


In [14]:
from tqdm import tqdm

In [15]:
references = []
hypotheses = []

for item in tqdm(dataset[n:]["translation"], desc="Translating"):
    en = item["en"]
    fa = item["fa"]

    pred = translate(en, icl_examples)

    references.append([fa])
    hypotheses.append(pred)

Translating: 100%|██████████| 95/95 [21:17<00:00, 13.45s/it]


In [1]:
print(f"{references[1]} \n {hypotheses[1]}")

NameError: name 'references' is not defined

## BLEU Score Evaluation

In [23]:
bleu = sacrebleu.corpus_bleu(hypotheses, references)
print(f"BLEU score: {bleu.score:.2f}")

BLEU score: 0.00
